In [8]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  2.4 ; cuda:  cpu


'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
import os
HOME = os.getcwd()
print(HOME)

D:\sanjay\Traffic_density_analysis_proj


In [10]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.27  Python-3.12.6 torch-2.4.1+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
Setup complete  (20 CPUs, 15.6 GB RAM, 0.2/146.5 GB disk)


In [11]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

from IPython import display
display.clear_output()


ModuleNotFoundError: No module named 'detectron2'

In [12]:
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

D:\sanjay\Traffic_density_analysis_proj


C:\Users\smsan\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
SOURCE_VIDEO_PATH = f"{HOME}/vehicle-counting.mp4"

In [14]:
!pip install supervision[assets] -q

from IPython import display
display.clear_output()

In [15]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

100%|█████████████████████████████████████████████████████████████████████████████| 21.5M/21.5M [04:31<00:00, 83.1kB/s]


In [16]:
import supervision as sv
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

Exception: Could not open video at D:\sanjay\Traffic_density_analysis_proj/vehicle-counting.mp4

In [ ]:
import numpy as np


# initiate polygon zone
polygon = np.array([
    [1700,600],
    [1400,600 ],
    [0,1800 ],
    [1900,1800 ]
])
zone = sv.PolygonZone(polygon=polygon)

# initiate annotators
box_annotator = sv.BoundingBoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.WHITE, thickness=6, text_thickness=6, text_scale=4)

# extract video frame
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
iterator = iter(generator)
frame = next(iterator)

# detect
results = model(frame, imgsz=1280)[0]
detections = sv.Detections.from_ultralytics(results)
x=zone.trigger(detections=detections)

# annotate
labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _, _ in detections if class_id in [2,3,5,7]]
frame = box_annotator.annotate(scene=frame, detections=detections)
frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)
frame = zone_annotator.annotate(scene=frame)

%matplotlib inline
sv.plot_image(frame, (16, 16))

In [ ]:
detections.xyxy.shape
area=0
for i in range(len(x)):
  if x[i]:
    area+=(detections.xyxy[i][2]-detections.xyxy[i][0])*(detections.xyxy[i][3]-detections.xyxy[i][1])
from shapely.geometry import Polygon
poly = Polygon(polygon)
density=round(area/poly.area*100,2)

In [ ]:
import numpy as np
from shapely.geometry import Polygon, box
from shapely.ops import unary_union

# Define the polygon zone
polygon = np.array([
    [1700, 600],
    [1400, 600],
    [0, 1800],
    [1900, 1800]
])
zone_polygon = Polygon(polygon)

# Initiate zone, annotators, and model
zone = sv.PolygonZone(polygon=polygon)

# Annotators
box_annotator = sv.BoundingBoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.WHITE, thickness=6, text_thickness=6, text_scale=4)

# Extract video frame
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
iterator = iter(generator)
frame = next(iterator)

# Detect objects
results = model(frame, imgsz=1280)[0]
detections = sv.Detections.from_ultralytics(results)
zone.trigger(detections=detections)

# Annotate labels
class_ids_of_interest = [2, 3, 5, 7]
labels = [
    f"{model.names[class_id]} {confidence:0.2f}"
    for _, _, confidence, class_id, _, _ in detections
    if class_id in class_ids_of_interest
]

# Annotate frame
frame = box_annotator.annotate(scene=frame, detections=detections)
frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)
frame = zone_annotator.annotate(scene=frame)

# Calculate area within polygon for specified class IDs
polygons = []
for (x_min, y_min, x_max, y_max, _, class_id) in detections:
    if class_id in class_ids_of_interest:
        # Create a bounding box polygon
        box_polygon = box(x_min, y_min, x_max, y_max)

        # Calculate the intersection of the bounding box with the zone polygon
        intersection = zone_polygon.intersection(box_polygon)

        if intersection.is_valid and not intersection.is_empty:
            polygons.append(intersection)

# Use unary_union to combine overlapping polygons
union_polygon = unary_union(polygons)

# Calculate the total area
total_area = union_polygon.area
print(f"Total bounded area for classes 2, 3, 5, 7: {total_area}")

# Plot the annotated frame
%matplotlib inline
sv.plot_image(frame, (16, 16))



0: 736x1280 6 cars, 1 truck, 1410.3ms
Speed: 12.7ms preprocess, 1410.3ms inference, 2.6ms postprocess per image at shape (1, 3, 736, 1280)
Bounding boxes within the polygon zone for classes 2, 3, 5, 7:


'box_annotator = sv.BoundingBoxAnnotator(thickness=4)\nlabel_annotator = sv.LabelAnnotator(text_thickness=4, text_scale=2)\nzone_annotator = sv.PolygonZoneAnnotator(zone=zone_polygon, color=sv.Color.WHITE, thickness=6, text_thickness=6, text_scale=4)\n\n# Annotate frame\nframe = box_annotator.annotate(scene=frame, detections=detections)\nframe = label_annotator.annotate(scene=frame, detections=detections)\nframe = zone_annotator.annotate(scene=frame)\n\n# Plot the annotated frame\n%matplotlib inline\nsv.plot_image(frame, (16, 16))\n'